In [1]:
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier

dataset_transfusion = pd.read_table('transfusion.data', sep=',')
print("\nDataset Transfusion:")
dataset_transfusion


Dataset Transfusion:


,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0
...,...,...,...,...,...
743,23,2,500,38,0
744,21,2,500,52,0
745,23,3,750,62,0
746,39,1,250,39,0


In [2]:
transfusion_normalised = dataset_transfusion.values.copy()
normalizador =  StandardScaler()

transfusion_normalised = normalizador.fit_transform(dataset_transfusion)
dataset_transfusion['Recency (months)'] = transfusion_normalised[:,0]
dataset_transfusion['Frequency (times)'] = transfusion_normalised[:,1]
dataset_transfusion['Monetary (c.c. blood)'] = transfusion_normalised[:,2]
dataset_transfusion['Time (months)'] = transfusion_normalised[:,3]

print("\nDataset Transfusion Normalized:")
dataset_transfusion


Dataset Transfusion Normalized:


,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,-0.927899,7.623346,7.623346,2.615633,1
1,-1.175118,1.282738,1.282738,-0.257881,1
2,-1.051508,1.796842,1.796842,0.029471,1
3,-0.927899,2.482313,2.482313,0.439973,1
4,-1.051508,3.167784,3.167784,1.753579,0
...,...,...,...,...,...
743,1.667904,-0.602307,-0.602307,0.152621,0
744,1.420685,-0.602307,-0.602307,0.727324,0
745,1.667904,-0.430940,-0.430940,1.137826,0
746,3.645659,-0.773675,-0.773675,0.193671,0


In [3]:
transfusion_values = dataset_transfusion.iloc[:,0:4].values
print("\nTransfusion valores dos atributos:\n")
print(transfusion_values)


Transfusion valores dos atributos:

[[-0.92789873  7.62334626  7.62334626  2.61563344]
 [-1.17511806  1.28273826  1.28273826 -0.2578809 ]
 [-1.0515084   1.79684161  1.79684161  0.02947053]
 ...
 [ 1.66790417 -0.43093957 -0.43093957  1.13782607]
 [ 3.64565877 -0.77367514 -0.77367514  0.19367135]
 [ 7.72477762 -0.77367514 -0.77367514  1.54832812]]


In [4]:
transfusion_classes = dataset_transfusion.iloc[:,4].values
print("\nTransfusion classes:\n")
print(transfusion_classes)
print("\nTransfusion classes shape:")
print(transfusion_classes.shape)


Transfusion classes:

[1 1 1 1 0 0 1 0 1 1 0 0 1 0 1 1 1 1 1 1 1 0 1 1 0 0 0 1 1 0 0 1 1 1 0 1 1
 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 1 1 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0
 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 1
 0 0 1 0 0 1 0 0 1 1 1 1 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 0 0 1
 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0
 0

In [5]:
kfold_treinamento = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=None) 
kfold_ajuste_parametros = model_selection.StratifiedKFold(n_splits=3, shuffle=True, random_state=None)

In [6]:
arvore_decisao = DecisionTreeClassifier()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo Arvore de decisao no dataset Transfusion ....\n")

param_dist = {'max_depth':[3,4,5,6,7,8,9,10], 'criterion': ['entropy'], 'splitter':['best','random']}
grid_search = GridSearchCV(arvore_decisao, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(transfusion_values, transfusion_classes)
decisionTreeBestParams = grid_search.best_params_
print("Decision Tree: %s \n\n" % decisionTreeBestParams)

====== Iniciando busca pelos melhores parâmetros do algoritmo Arvore de decisao no dataset Transfusion ....

Decision Tree: {'criterion': 'entropy', 'max_depth': 3, 'splitter': 'random'} 




In [7]:
knn_vizinhos_proximos = KNeighborsClassifier()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo KNN no dataset Transfusion ....\n")

param_dist = {'n_neighbors': list(np.arange(1, 10)), 'metric':['euclidean'], 'weights':['uniform', 'distance']} #Parâmetros testados
grid_search = GridSearchCV(knn_vizinhos_proximos, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(transfusion_values, transfusion_classes)
knnBestParams = grid_search.best_params_
print("KNN: %s \n\n" % knnBestParams)

====== Iniciando busca pelos melhores parâmetros do algoritmo KNN no dataset Transfusion ....

KNN: {'metric': 'euclidean', 'n_neighbors': 4, 'weights': 'uniform'} 




In [8]:
regressao_logistica = LogisticRegression()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo Regressao logistica no dataset Transfusion ....\n")

param_dist = {'penalty':['l2','none'],'tol':[0.0001,0.001, 0.01, 0.1, 1], 'max_iter':[300,400,500,600],'solver':['lbfgs','saga']} #Parâmetros testados

grid_search = GridSearchCV(regressao_logistica, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(transfusion_values, transfusion_classes)
regressao_logistica_param = grid_search.best_params_
print("Regressão logistica: %s \n\n" % regressao_logistica_param)


====== Iniciando busca pelos melhores parâmetros do algoritmo Regressao logistica no dataset Transfusion ....

Regressão logistica: {'max_iter': 400, 'penalty': 'none', 'solver': 'saga', 'tol': 1} 




In [ ]:
rede_neural = MLPClassifier()
print("====== Iniciando busca pelos melhores parâmetros do algoritmo Redes neurais no dataset Transfusion ....\n")

param_dist = {'hidden_layer_sizes':list(np.arange(5, 10)),'activation':['logistic'], 'max_iter':[300,400,500,600],'alpha':[0.0001,0.001, 0.01, 0.1], 'solver':['sgd','adam'], 'learning_rate':['constant','invscaling','adaptive'], 'learning_rate_init':[0.001, 0.01, 0.1]} #Parâmetros testados

grid_search = GridSearchCV(rede_neural, param_grid=param_dist, cv=kfold_ajuste_parametros, scoring='accuracy', refit=False)
grid_search.fit(transfusion_values, transfusion_classes)
redes_neurais_param = grid_search.best_params_
print("Redes neurais: %s \n\n" % redes_neurais_param)

====== Iniciando busca pelos melhores parâmetros do algoritmo Redes neurais no dataset Transfusion ....



/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/anapedroso/Documentos/IA-desempenho-algoritmos-classificacao/env/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_pe

In [ ]:
predicted_classes = dict()
predicted_classes['Arvore_decisao'] = np.zeros(transfusion_classes.shape)
predicted_classes['KNN_Vizinhos_proximos'] = np.zeros(transfusion_classes.shape)
predicted_classes['Naive_bayes_gaussian'] = np.zeros(transfusion_classes.shape)
predicted_classes['Regressao_logistica'] = np.zeros(transfusion_classes.shape)
predicted_classes['Rede_neural'] = np.zeros(transfusion_classes.shape)

arvore_decisao_best_param = DecisionTreeClassifier(**decisionTreeBestParams)
knn_vizinhos_proximos_best_param = KNeighborsClassifier(**knnBestParams)
naive_bayes_gaussian = GaussianNB()
regressao_logistica_best_param = LogisticRegression(**regressao_logistica_param)
rede_neural_best_param = MLPClassifier(**redes_neurais_param)

In [ ]:
for train, test in kfold_treinamento.split(transfusion_values, transfusion_classes):
    data_train, target_train = transfusion_values[train], transfusion_classes[train]
    data_test, target_test = transfusion_values[test], transfusion_classes[test]

    arvore_decisao_best_param = arvore_decisao_best_param.fit(data_train, target_train)
    arvore_decisao_predicted = arvore_decisao_best_param.predict(data_test)
    predicted_classes['Arvore_decisao'][test] = arvore_decisao_predicted

    knn_vizinhos_proximos_best_param = knn_vizinhos_proximos_best_param.fit(data_train, target_train)
    vizinhos_proximos_predicted = knn_vizinhos_proximos_best_param.predict(data_test)
    predicted_classes['KNN_Vizinhos_proximos'][test] = vizinhos_proximos_predicted

    naive_bayes_gaussian = naive_bayes_gaussian.fit(data_train, target_train)
    naive_bayes_gaussian_predicted = naive_bayes_gaussian.predict(data_test)
    predicted_classes['Naive_bayes_gaussian'][test] = naive_bayes_gaussian_predicted

    regressao_logistica_best_param = regressao_logistica_best_param.fit(data_train, target_train)
    regressao_logistica_predicted = regressao_logistica_best_param.predict(data_test)
    predicted_classes['Regressao_logistica'][test] = regressao_logistica_predicted

    rede_neural_best_param = rede_neural_best_param.fit(data_train, target_train)
    rede_neural_predicted = rede_neural_best_param.predict(data_test)
    predicted_classes['Rede_neural'][test] = rede_neural_predicted

In [ ]:
for classificador in predicted_classes.keys():
    print("================================================================================================")
    print("Resultados do classificador %s\n%s\n"
          %(classificador, metrics.classification_report(transfusion_classes, predicted_classes[classificador])))
    print("Matriz de confusão: \n%s\n\n\n" % metrics.confusion_matrix(transfusion_classes, predicted_classes[classificador]))